## IMBD_review
-This project takes a list of opinions on IMBD rated as: good(1) or bad(0). With them he trains the words to divide them into good or bad perception. 

-Este proyecto toma una lista de opiniones en IMBD calificadas como: buenas ó malas. Con ellas entrena las palabras para dividirlas  entre percepción buena ó mala.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
import numpy as np

In [2]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [3]:
# decode the train,test and label
train_data,test_data= imdb["train"],imdb["test"]
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
    training_sentences.append(s.numpy().decode("utf8"))
    training_labels.append(l.numpy())
for s,l in train_data:
    testing_sentences.append(s.numpy().decode("utf8"))
    testing_labels.append(l.numpy())
    
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)
print(len(testing_labels_final))

25000


In [4]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer= Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index= tokenizer.word_index

sequences= tokenizer.texts_to_sequences(training_sentences)
padded= pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)
#example:
testing_padded

array([[   0,    0,    0, ...,  867,  141,   10],
       [   0,    0,    0, ...,   20,   31,   30],
       [1341,    3, 2339, ...,  417,    9, 6110],
       ...,
       [  34,   30, 1815, ...,   16,   12,   29],
       [ 793, 1073,    6, ...,   78, 1290,   23],
       [ 222,    9,   50, ...,   32, 1663,  709]])

In [5]:
# back to see how the phrase looks. 
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [6]:
#Model:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [7]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.5073 - accuracy: 0.7304 - val_loss: 0.3091 - val_accuracy: 0.8709
Epoch 2/10
782/782 [==============================] - 4s 6ms/step - loss: 0.2435 - accuracy: 0.9059 - val_loss: 0.2836 - val_accuracy: 0.8835
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0939 - accuracy: 0.9766 - val_loss: 0.3073 - val_accuracy: 0.8817
Epoch 4/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0238 - accuracy: 0.9971 - val_loss: 0.3549 - val_accuracy: 0.8799
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0064 - accuracy: 0.9996 - val_loss: 0.3946 - val_accuracy: 0.8801
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0021 - accuracy: 0.9999 - val_loss: 0.4293 - val_accuracy: 0.8815
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 9.2243e-04 - accuracy: 1.0000 - val_loss: 0.4564 - val_accuracy: 0.8811
Ep

In [8]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [9]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [10]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [11]:
# try your sentences and look how it is.

sentence = "This is amazing,really"
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[12, 7, 478, 64]]
